### A.6 Functions For DBSCAN Clustering Algorithms

In [1]:

# ==================================================================================

def get_cluster_label_dbscan(X, para):
    
    """ 
    Return the results (cluster labels) of each data points using DBSCAN clustering algorithm
        
        
    Parameters:
    -----------
    
    X: an array shape: (n_samples, n_attributes)
        It is data matrix
        
        
    para: a dictionary 
        It contains the parameters of the clustering algorihtms    
        
           
        
    Return:
    -------
    
    label_predicted: an array shape: n_samples
    
        It is the labels assigned by the Kmean clustering algorithm
    
    
    """
    # Create an object of dbsca  clustering and fit the model
    c_obj = cluster.DBSCAN(eps=para['eps'],
                           min_samples = para['min_samples'],
                           metric = para['metric']).fit(X) # no random_state
    # get the predicted labels
    if hasattr(c_obj, 'labels_'):
        label_predicted = c_obj.labels_.astype(np.int)
    else:
        label_predicted = c_obj.predict(X)
    
    return label_predicted
    

# ==================================================================================

# def get_parameter_dbscan(data, eps_range=(0.1, 7), min_point_range=(2, 30), total_point=50):
# 7th August
def get_parameter_dbscan(data, eps_range=(0.1, 2), min_point_range=(3, 6), total_point=50, display_results=False):
    
    """ 
    Return a dictionary containing the optimal values of parameter for DBSCAN algorithm
    based on Silhouette score and a tuple of Silhouette metrics
    
    Parameters:
    -----------
    
    data:  a tuple
    
    The first element, X, is a 2 dimensional array of shape 
    (n_samples, n_attributes)and the second element, labels of clusters, is a one dimensional 
    array of shape (n_samples) representing the cluster label of individual points. 
    
    
    eps_range: a tuple
    
        It holds the range of values for the parameter 'eps' (a radius) for the DBSCAN. 
        
    min_point_range: a tuple
    
        It holds the range of values for the parameter 'min_samples' (n neighbours), for the DBSCAN. 
        
        a heurestic guideline (default minPnt = 2* dimension of the dataset) from the paper below:
        
        Jörg Sander, Martin Ester, Hans-Peter Kriegel, and Xiaowei Xu. 1998. Density-based clustering in spatial databases:
The algorithm GDBSCAN and its applications. Data Mining and Knowledge Discovery 2, 2 (1998), 169–194. DOI:http:
//dx.doi.org/10.1023/A:1009745219419
        
    total_point: a scalar
    
        It represents how many 'eps' values to be used for optimization purposes. 
        
    
    display_results: a boolean
        True indicates to print the outputs of the optimization and False supresss the print
        
        
    Return:
    -------
    
    opt_para, (score_l2, score_l1, score_cosine): a tuple of two elements    
    
        opt_para: a dictionary 
        
        It holds the parameters for DBSCAN clustering algorithm that is optimized for the 
        dataset (data) using Silhouette scores.
        
        (score_l2, score_l1, score_cosine): a tuple
        
        From left to right, these are three one-dimensional arrays containing average Silhouette scores for 
        different values of the parameter(s) computed using 'euclidean', 'manhattan', and 'cosine' 
        distance metrics respectively .
        
      
    """
    
    # eps_range and min_point_range are tuple of two elements
    
    X, y_true = data
    
#     increment = 3
# 7 August
    increment = 1
    
    # define ranges of parameters
    eps = np.linspace(start= eps_range[0], stop=eps_range[1], num = total_point)
    min_pt = range(min_point_range[0], min_point_range[1], increment)
    distance_metric = ['euclidean']
    
    # create grid of parameters
    param_grid = list(model_selection.ParameterGrid({'eps': eps, 
                                                'min_samples': min_pt, 
                                                'metric':distance_metric}
                                              )
                     )
# -----------------------------
    if display_results:
        print()
        print("Optimizing Parameters: DBSCAN =====")
        print()
        print("Total Parameter Settings: ", len(param_grid))
# -----------------------------    
    score_l2, score_l1, score_cosine = [], [], []
    
    for i, para in enumerate(param_grid):
        
        y_pred = get_cluster_label_dbscan(X, para)    
        s_l2, s_l1, s_cosine = get_performance_metrics(X, y_true, y_pred)
        
        score_l2.append(s_l2)
        score_l1.append(s_l1)
        score_cosine.append(s_cosine)
        
        # create data frame to inlcude silhouette score including the parameter grid
        param_grid[i]['score_l2'] = s_l2
# ----------------------------------------------------        
        if i == 0:
            df = pd.DataFrame([para])
        else:
            df = df.append(pd.DataFrame([para]), ignore_index=True)

# --------------------------------------------------------------       

    #  get optimal parameters based on metric       

    index_l2  = np.argmax(score_l2)
#     index_l1  = np.argmax(score_l1)
#     index_cosine  = np.argmax(score_cosine)
    
    
    
#     # check if multiple indices haves the same max values
#     max_val_l2 = score_l2[index_l2]
#     indices = [i for i, v in enumerate(score_l2) if v == max_val_l2]
#     max_grid_index = np.random.choice(indices)
    
#     opt_para = param_grid[max_grid_index]
    
    opt_para = param_grid[index_l2]
         
#     print()
#     print('DBSCAN---')
#     print()
    
#     if len(np.unique([index_l2, index_l1, index_cosine])) == 1:
#         print("Maximum Silhouette Scores (l2, l1, and cosine) at Same Index!!!")
#     else:
#         print("Maximum Silhouette Scores (l2, l1, and cosine) at Different Index!!!")
        
#     print('Indices at Maximum Silhouette Scores (l2): ', indices) 
#     print()
#-------------------------------------------
#     if display_results:
#         print('{:<15s}:  eps = {:.5f}, min_samples = {}, metric = {}'.format('DBSCAN:', opt_para['eps'], opt_para['min_samples'], opt_para['metric']))
    
#         print()
#         print("Summary: Maximum Silhouette Scores At Parameters ==== ")
#         #  summarize the result of optimization in a data frame
#-------------------------------------------
    if display_results: 
        print()
        for i in range(min_point_range[0], min_point_range[1], increment):
            # get the index at maximum silhouette score
            ind = int(df.loc[lambda d: d.min_samples == i, ['score_l2']].idxmax())
            if i == min_point_range[0]:
                new_df = df.loc[lambda d: d.index == ind, :]
            else:
                new_df = new_df.append(df.loc[lambda d: d.index == ind, :], ignore_index=True)

        print()
        print(new_df)
  
    
    
#     # display silhouette graph 
#     cluster_labels = get_cluster_label_dbscan(X, opt_para)
#     silhouette_plot(X, cluster_labels)
# # ----------------------  
    
    
    return opt_para, (score_l2, score_l1, score_cosine)


# ==================================================================================

def knee_plot_knn(test_samples, min_sample=[3]):    
    """
    Plot knee graph based on average k-nn distances 
    
    Parameters:
    ----------
    
    test_samples:  a tuple
    
    The first element, data matrix, is a 2 dimensional array of shape 
    (n_samples, n_attributes)and the second element, clusters labels, is a one dimensional 
    array of shape (n_samples) representing the cluster labels of individual points. 
    
    min_sample: an array of one dimension
        Each element in this array represents the number of nearest neighbours
        
    Return:
    ------
    
    None
    """
    X, y = test_samples
    
    # with n_neighbours = 1, self neighbours so distance is zero. 
    
    n_fig = len(min_sample) + 1
#     w, h = 3.5, 3
    # fix the size of the figure window    
    if n_fig < 7:
        n_col = 6
        n_row = 1
    elif n_fig < 13:
        n_col = 6
        n_row = 2
    elif n_fig < 19:
        n_col = 6
        n_row = 3
            
    fig = plt.figure(figsize=(f_w*n_col, f_h*n_row))  
    fig.subplots_adjust(wspace=0.35)
    
    fig_num = 1
    plt.subplot(n_row, n_col,fig_num)    
    plot_dataset(test_samples, "Original Data")
    for m_sample in min_sample:
        fig_num +=1                
        if fig_num <= n_fig and m_sample <= X.shape[0]:        
            # fit the model with requried knn
            model = NearestNeighbors(n_neighbors=m_sample,
                                     metric='minkowski',
                                     p=2).fit(X)
            dist, idx = model.kneighbors(X)
            # compute k-nn average distance of each point
            mean_dist = np.mean(dist,axis=1)
            mean_dist_sort = mean_dist[np.argsort(mean_dist)]
            # compute where large difference occures
            dist_diff = []
            for i in range(len(mean_dist_sort)-1):
                dist_diff.append(mean_dist_sort[i+1] - mean_dist_sort[i])
            # max distance differentc at index
            eps = mean_dist_sort[np.argmax(dist_diff)]
            # print('Mean average distance', mean_dist)
            # plotting        
            
            plt.subplot(n_row, n_col, fig_num)
            plt.plot(mean_dist_sort, '-g.' , lw=0, markersize = 4)
        #     plt.hlines(y = eps, 
        #                xmin=0, xmax=X.shape[0], 
        #                label= 'eps', 
        #                color= 'red',
        #                linestyles= 'dashdot')

        #     plt.yticks([0.1, 0.2, 0.3, 0.4 , 0.5, 0.6 , 0.7, 0.8])
        #     plt.xticks([0, 15, 30, 45 , 60, 75, 90, 100])
            plt.xlabel('Points sorted by distance')
            plt.ylabel('{}-NN distance'.format(m_sample))
            plt.grid()
    return None


def knee_plot_dbscan(test_samples, knn=3, ax=None):    
    """
    Plot knee graph given the knn based on average k-nn distances 
    
    Parameters:
    ----------
    
    test_samples:  a tuple
    
    The first element, data matrix, is a 2 dimensional array of shape 
    (n_samples, n_attributes)and the second element, clusters labels, is a one dimensional 
    array of shape (n_samples) representing the cluster labels of individual points. 
    
    knn: a scalar
        The number represents the number of nearest neighbours
        
    Return:
    ------
    
    None
    """
    X, y = test_samples
    
    ax = ax or plt.gca()
    
    # with n_neighbours = 1, self neighbours so distance is zero. 
       
    
    
    
    if knn <= X.shape[0]:        
        # fit the model with requried knn
        model = NearestNeighbors(n_neighbors=knn,
                                 metric='minkowski',
                                 p=2).fit(X)
        dist, idx = model.kneighbors(X)
        # compute k-nn average distance of each point
        mean_dist = np.mean(dist,axis=1)
        mean_dist_sort = mean_dist[np.argsort(mean_dist)]

    ax.plot(mean_dist_sort, '-g.' , lw=0, markersize = 4)

    ax.set_xlabel('Points sorted by distance')
#     ax.set_ylabel('{}-NN distance'.format(knn))   
#     ax.axis('equal')
        
    return None


# ==================================================================================

def display_dbscan_outputs(test_samples, para_dbscan):
    
    '''
    Dispaly the outputs of the DBSCAN clustering algorithm and Silhouette plot
    
    Parameters: 
    -----------
    
    and the second element, clusters labels, is a one dimensional 
    array of shape (n_samples) representing the cluster labels of individual points. 
        
    
    para_dbscan: a dictionary
    
        It contains the parameters of dbscan algorihtms. It looks like 
        para_dbscan = { 'eps': 0.5, 'min_samples': 5, 'metric':'euclidean'}
        
    Return:
    ------
    
    result: a dictionary
        It holds the results or attributes of DBSCAN object.
    
    '''
    
        
    X, y = test_samples
    
#     print('Data points =====')
#     print(X)
    # pairwise_distances(test_samples[0], metric='l2')
    # initialize the algorithm with parameters
    print("Parameters =====")
    print('eps = {:.4f}, min_samples = {}, metric = {}'.\
          format(para_dbscan['eps'], para_dbscan['min_samples'], para_dbscan['metric']))
   
    t0 = time.time()
    model = cluster.DBSCAN(eps = para_dbscan['eps'],
                           min_samples = para_dbscan['min_samples'],
                           metric = para_dbscan['metric'])
       
    # fit the data to the model
    model.fit(X)
    t1 = time.time()
    
    n_col = 4
    n_row = 1
    
    plt.figure(figsize=(n_col * f_w, n_row * f_h))
    
    plt.subplot(1, 4, 1)
    plot_dataset(test_samples, "Original Data")
        
    plt.subplot(1, 4, 2)
    ax2 = plt.gca()
    knee_plot_dbscan(test_samples, para_dbscan['min_samples'], ax=ax2)  
    
    ax3 = plt.subplot(1, 4, 3)
    ax4 = plt.subplot(1, 4, 4)   
    
    silhouette_plot(X, model.labels_, (ax3, ax4), t1-t0)

    result =  {'labels':model.labels_,
              'core_sample_indices':model.core_sample_indices_,
              'core_samples': model.components_,
              'tot_time': t1-t0}
    
    return result



def test_dbscan_varying_minPnt(X, minPnt_range, test_para):
    """ Plot the result of DBSCAN by varing the n_neighbour parameter and keeping eps or radious fixed
    
    Parameter:
    ---------
    
    X: an array
    
    The data matrix, X, is a 2 dimensional array of shape (n_samples, n_attributes)
    
    minPnt_range: a tuple
        It holds the range of number of neighbours 
    
    test_para: a dictionary
        It holds the parameters of DBSCAN algorithm
        
    Return:
        None
    
    """
    for minPnt in range(minPnt_range[0], minPnt_range[1]):
        test_para['min_samples'] = minPnt
#         print(test_para)
        print_parameters(test_para)
        label = get_cluster_label_dbscan(X, test_para)
        silhouette_plot(X, label)
        c_label, cluster_size = check_cluster_size(label)
        print('-----------------------')
        print("Total cluster = {} \t Cluster size {}\n".format(len(c_label), cluster_size))
    
    return None


def test_dbscan_varying_eps(X, eps_range, test_para):
    """ Plot the result of DBSCAN by varing the eps parameter and keeping n_neighbour fixed
    
    Parameter:
    ---------
    
    X: an array
    
    The data matrix, X, is a 2 dimensional array of shape (n_samples, n_attributes)
    
    eps_range: a tuple
        It holds the range of radius. 
    
    test_para: a dictionary
        It holds the parameters of DBSCAN algorithm.
        
    Return:
        None
    
    """
    for eps in np.linspace(eps_range[0], eps_range[1], 20, endpoint=True):
        test_para['eps'] = eps
        #         print(test_para)
        print_parameters(test_para)
        label = get_cluster_label_dbscan(X, test_para)
        silhouette_plot(X, label)
        c_label, cluster_size = check_cluster_size(label)
        print('-----------------------')
        print("Total cluster = {} \t Cluster size {}\n".format(len(c_label), cluster_size))
    
    return None

    